## Тестовое задание
### Воротников Владимир


### 1. Знакомство с API <br>
Получим информацию о POI в ручном режиме

In [1]:
# !pip install openrouteservice ortools folium

In [36]:
import folium
from openrouteservice import client

Получим пустую карту интересующего нас места и убедимся, что заданы правильные координаты

In [37]:
api_key = '5b3ce3597851110001cf62482ccff5e54ea24de79d108a5cb78f7d78'
clnt = client.Client(key=api_key)

# в качестве координат использую свои
my_loc = ([55.998976, 37.219673])
map_1 = folium.Map(tiles='Stamen Toner', location=my_loc, zoom_start=15)
map_1

Зададим и отобразим на карте интересующую нас область

In [38]:
params_iso = {'range_type': 'distance',
              'range': [1000],
             }
params_iso['locations'] = [list(reversed(my_loc))]
my_region = {}
my_region['iso'] = clnt.isochrones(**params_iso)
folium.features.GeoJson(my_region['iso']).add_to(map_1)
map_1

Найдем интересующие нас объекты (банкоматы, супермаркеты) в ранее заданной области

In [39]:
params_poi = {'request': 'pois',
              'sortby': 'distance'}

categories_poi = {'atm': [191],
                  'supermarket': [518]}

my_region['categories'] = dict()
params_poi['geojson'] = my_region['iso']['features'][0]['geometry']
print("My Region")
    
for typ, category in categories_poi.items():
    params_poi['filter_category_ids'] = category
    my_region['categories'][typ] = dict()
    my_region['categories'][typ]['geojson']= clnt.places(**params_poi)[0]['features']
    print(f"\t{typ}: {len(my_region['categories'][typ]['geojson'])}")

My Region
	atm: 3
	supermarket: 5


In [40]:
poi_coords = {}

for cat, pois in my_region['categories'].items():
    for poi in pois['geojson']:
        if cat not in poi_coords:
            poi_coords[cat] = []
        poi_coords[cat].append(poi['geometry']['coordinates'])

poi_coords

{'atm': [[37.212384, 56.001591],
  [37.215998, 55.996413],
  [37.21567, 55.995869]],
 'supermarket': [[37.210458, 55.997989],
  [37.209, 56.000198],
  [37.216219, 55.996584],
  [37.218563, 55.996989],
  [37.213194, 55.993363]]}

Отобразим найденные POI на карте

In [41]:
style_dict = {'atm': 'credit-card',
              'supermarket': 'shopping-cart',
             }

for cat, pois in poi_coords.items():
    for poi in pois:
        folium.map.Marker(list(reversed(poi)),
                                      icon=folium.Icon(color='white',
                                                       icon_color='#1a1aff',
                                                       icon=style_dict[cat],
                                                       prefix='fa'
                                                      )
                                     ).add_to(map_1)
map_1


Также мы можем сохранить полученные данные в HTML или вернуть HTML-текст (это будет полезно при создании веб-приложения, выдающего результаты онлайн)

In [34]:
map_1.save('map.html')
# map_1._repr_html_()

### 2. Веб-приложение
Соберём поиск и отображение POI в одну функцию, которую в дальнейшем будем использовать в веб приложении<br>
Функция будет получать на вход координаты, область поиска, категорию интересующих объектов<br>
На выходе она будет формировать HTML с картой и отраженными на ней POI<br>

In [29]:
api_key = 'deleted'

def find_pois(location: list, distance_range: int, poi_category: dict) -> str:
    '''
    Args:
        location (list): coordinates of initial location (lat, lon)
        distance_range (int): distance from initial locaton
        poi_category (dict): poi categories according to https://github.com/GIScience/openrouteservice-docs#places-response
    
    Args example:
        location = [55.998976, 37.219673]
        distance_range = 1000
        poi_category = {'atm': 191}
    
    Returns:
        str: containing html-code with a map and poi-markers on it 
    '''
    location = (location)
    clnt = client.Client(key=api_key)
    map_1 = folium.Map(tiles='Stamen Toner', location=(location), zoom_start=15)
    params_iso = {'range_type': 'distance',
                  'range': [distance_range],
                 }
    params_iso['locations'] = [list(reversed(location))]
    my_region = dict()
    my_region['iso'] = clnt.isochrones(**params_iso)
    folium.features.GeoJson(my_region['iso']).add_to(map_1)
    
    params_poi = {'request': 'pois',
              'sortby': 'distance'}

    my_region['categories'] = dict()
    params_poi['geojson'] = my_region['iso']['features'][0]['geometry']

    # for now, frontend supports only 1 category per request, but backend is ready to work with a multiply categories
    for typ, category in poi_category.items():
        params_poi['filter_category_ids'] = category
        my_region['categories'][typ] = dict()
        my_region['categories'][typ]['geojson']= clnt.places(**params_poi)[0]['features']
    
    poi_coords = {}

    for cat, pois in my_region['categories'].items():
        for poi in pois['geojson']:
            if cat not in poi_coords:
                poi_coords[cat] = []
            poi_coords[cat].append(poi['geometry']['coordinates'])

    for cat, pois in poi_coords.items():
        for poi in pois:
            folium.map.Marker(list(reversed(poi)),
                                          icon=folium.Icon(color='white',
                                                           icon_color='#1a1aff',
                                                           icon='angle-down',
                                                           prefix='fa'
                                                          )
                                         ).add_to(map_1)
    return map_1

In [32]:
location = [55.998976, 37.219673]
distance_range = 1000
poi_category = {'atm': [191]}
map_2 = find_pois(location, distance_range, poi_category )
map_2

Имея данную функцию мы уже можем строить наше веб-приложение